In [14]:
# https://ocrapi.visive.ai/

# Post - form data - key: file , value : pdf


In [51]:
res = {
    "detected_text": [
        "(PLEASE WRITE IN BLOCK LETTERS)",
        "Name",
        "ESAKKY MUTHU.M",
        "Door No",
        "14/11",
        "Address",
        "ACHUTANNAGAR",
        "EKKATUTHANGAL",
        "City",
        "CHENN",
        "-",
        "Pincode",
        "600032",
        "Taluk",
        "District",
        "CHENNAI",
        "Mobile",
        "6381683601",
        "Landline",
        "e-mail",
        "ESAKKYMUTHU403@GMAIL.COM",
        "(PLEASE WRITE IN BLOCK LETTERS)",
        "Name",
        "SELVAGANESH.S",
        "Door No",
        "S 2",
        "Address",
        "2NDCROSSSTREET BALAJI",
        "NAGAR ERKATUTHANGAL",
        "City",
        "CHENNAI",
        "Pincode",
        "600cB2",
        "1",
        "Taluk",
        "District",
        "CHENNAI",
        "Mobile",
        "6382669478",
        "Landline",
        "e-mail",
        "SSELVAGANESH14102000@gm,l.com",
        "-"
    ],
    "status": "success"
}



In [52]:
res2 = {'status': 'success', 'detected_text': ['.',
                                               '(PLEASE WRITE IN BLOCK LETTERS)', 'Name', 'ESAKKY MUTHU.M', 'Door No', '14/11', 'Address', 'ACHUTANNAGAR',
                                               'EKKATUTHANGAL', 'City', 'CHENNAI', 'Denomination :', '2000 x', '1', 'Pincode', '600032', '500 x', '=', 'Taluk', '200 x=',
                                               '100 x', '=', 'District', 'CHENNAI', '50 x', '=', 'Mobile', '6381683601', '20 x', '10 x', '=', 'Landline', 'Total',
                                               '=', 'e-mail', 'ESAKKYMUTHU403@GMAIL.COM', '(PLEASE WRITE IN BLOCK LETTERS)', 'Name', 'SELVAGANESH.S', 'Door No', 'S 2',
                                               'Address', '2NDCROSSSTREET BALAJI', 'NAGAR ERKATUTHANGAL', 'City', 'CHENNAI', 'Denomination :', '2000 x', '-', '1',
                                               'Pincode', '600032', '500 x', 'Taluk', '200 x', '=', '100 x', 'District', 'CHENNAI', '50 x', '-', 'Mobile',
                                               '6382669478', '20 x', '10>', '-', 'Landline', 'Total', '=', 'e-mail', 'SSELVAGANESH14102000@gmail.com', '-']}


In [53]:
#from __future__ import division
import re
from collections import Counter
from email_validator import validate_email,EmailNotValidError, ValidatedEmail
from string import punctuation
import pandas as pd                                                                        

In [66]:
def words(text):
    return re.findall(r'\w+', text.lower())

WRDS= Counter(words(open("cities.txt").read()))
WRD= Counter(words(open("email_new.txt").read()))

def P(word, N=sum(WRDS.values())):
    #"Probability of `word`."
    return WRDS[word] / N
def R(word, N=sum(WRD.values())):
    #"Probability of `word`."
    return WRDS[word] / N

def candidates(word,WORDS):
    "Generate possible spelling corrections for word."
    if set([w for w in [word] if w in WORDS]):
        return set([w for w in [word] if w in WORDS])
    else:
        result_edits1 = edits1(word)
        known_result_edits1 = set([w for w in result_edits1 if w in WORDS])

        if (known_result_edits1):
            return known_result_edits1
        else:
            result_edits2 = edits2(result_edits1)
            known_result_edits2 = set([w for w in result_edits2 if w in WORDS])

            if (known_result_edits2):
                return known_result_edits2
            else:
                result_edits3 = edits3(result_edits2)
                known_result_edits3 = set([w for w in result_edits3 if w in WORDS])
                if (known_result_edits3):
                    return known_result_edits3
                else:
                    return [word]

def edits1(word):
    #"All edits that are one edit away from `word`."
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts = [L + c + R for L, R in splits for c in letters]
    final_added_list = deletes + transposes + replaces + inserts
    return set(final_added_list)

def edits2(result_edits1):
    "All edits that are two edits away from `word`."
    return [e2 for e1 in result_edits1 for e2 in edits1(e1)]


def edits3(result_edits2):
    print("System gone in deep to answer your query better. Please be Patient.  :")
    return [e3 for e2 in result_edits2 for e3 in edits1(e2)]


In [67]:
d1={'0':['a','C','c','D','d','e','n','o','O','p','P','Q','u','U'], 
'1':['i', 'I', 'j', 'J', 'L', 'l', 'r','t'], 
'2':['z','Z'], 
'3':['E'], 
'4':['A','H','h', 'K', 'k','m','M', 'N','R','v', 'V','w','W',], 
'5':['S', 's'], 
'6':['b','G'], 
'7':['T','f','F'], 
'8':['B','x','X'], 
'9':['g','q','Y','y']}

In [69]:
def check_email(email):
    try:
      # validate and get info
        v = validate_email(email)
        # replace with normalized form
        email = v["email"] 
        print('ok_email',email)
    except EmailNotValidError as e:
        # email is not valid, exception message is human-readable
        if re.search(".com$", email):
          ini=email.split('.')[0]
          em_pro=ini.split('@')[-1].lower()
          name_id=ini.split('@')[0]
          print(em_pro)
          em_pro_cor=str(max(candidates(em_pro,WRD),key=R))
          email=name_id+'@'+ em_pro_cor + '.com'
          print('correction emial',em_pro_cor)
        if re.search(".in$", email):
          ini=email.split('.')[0]
          em_pro=ini.split('@')[-1].lower()
          name_id=ini.split('@')[0]
          print(em_pro)
          em_pro_cor=str(max(candidates(em_pro,WRD),key=R))
          email=name_id+'@'+ em_pro_cor + '.in'
          print('correction emial',em_pro_cor)
    return email

In [59]:
def cheak_pincode(pincode):
    if pincode.isalnum():
        for i in pincode:
            if i.isalpha():
                num=[j for j, s in enumerate(list(d1.values())) if i in s]
                # print(num)
                ind=pincode.index(i)
                pincode=pincode.replace(pincode[ind],str(num[0]))
                # print(pincode)
    return pincode

In [60]:
fields = ['Name', 'Door No', 'Address', 'City', 'Pincode',
          'Taluk', 'District', 'Mobile', 'Landline', 'e-mail']
ocr_json = res['detected_text']
def process_text1(ocr_json, fields):
    df = pd.DataFrame(columns=fields)
    all_info = {}
    ind = ocr_json.index('Name')
    ocr_json = ocr_json[ind:]
    raw_text = ' '.join(ocr_json)
    raw_text = re.sub("(PLEASE WRITE IN BLOCK LETTERS)", "",
                      raw_text).replace("()", "").strip()
    raw_text = re.sub("Denomination : 2000 x", "", raw_text)
    print('raw_text', raw_text)
    # raw_text = re.sub("Denomination : 2000 x","",raw_text)
    # raw_text =re.sub("[^A-Z]", "", raw_text,0,re.IGNORECASE)

    d = 'Name'
    records = [d+e for e in raw_text.split(d) if e]
    print('records', records)
    for j, record in enumerate(records):
        p_info = {}
        for i, field in enumerate(fields[:-1]):
            p_info['Name'] = (record.split('Name'))[1].split(
                'Door No')[0].strip().strip(punctuation).strip()
            p_info['Door No'] = (record.split('Door No'))[1].split(
                'Address')[0].strip().strip(punctuation).strip()
            p_info['Address'] = (record.split('Address'))[1].split('City')[
                0].strip().strip(punctuation).strip()
            p_info['City'] = (record.split('City'))[1].split(
                'Pincode')[0].strip().strip(punctuation).strip()
            p_info['Pincode'] = (record.split('Pincode'))[1].split(
                'District')[0].strip().strip(punctuation).strip()
            p_info['District'] = (record.split('District'))[1].split(
                'Mobile')[0].strip().strip(punctuation).strip()
            p_info['Mobile'] = (record.split('Mobile'))[1].split(
                'Landline')[0].strip().strip(punctuation).strip()
            p_info['Landline'] = (record.split('Landline'))[1].split(
                'e-mail')[0].strip().strip(punctuation).strip()
            p_info['e-mail'] = (record.split('e-mail')
                                )[1].strip().strip(punctuation).strip()

        if (len(p_info['Pincode']) > 6 and " " in p_info['Pincode']):
            p_info['Pincode'] = p_info['Pincode'].split(" ")[0]
        city_name=p_info['City'].lower()
        # print(str(correction(city_name)))
        p_info['City']= str(max(candidates(city_name,WRDS),key=P))
        p_info['City']=p_info['City'].upper()
        email=p_info['e-mail']
        email=check_email(email)
        p_info['e-mail']= email
        pincode=p_info['Pincode']
        pincode=cheak_pincode(pincode)
        p_info['Pincode']=pincode
        mobile_num=p_info['Mobile']
        mobile_num=cheak_pincode(mobile_num)
        p_info['Mobile']=mobile_num  
        
        all_info[j] = p_info
    
        #temp_df = pd.DataFrame(p_info)
        temp_df = pd.Series(p_info).to_frame().transpose()
        df = pd.concat([df, temp_df], ignore_index=True)

    df.columns = df.columns.str.replace(' ', '_')
    # xml_response = df.to_xml()
    # df.to_csv('test.csv')
    #df = df.join(temp_df)
    return all_info, df


In [61]:
res2 = {'status': 'success', 'detected_text': ['.',
                                               '(PLEASE WRITE IN BLOCK LETTERS)', 'Name', 'ESAKKY MUTHU.M', 'Door No', '14/11', 'Address', 'ACHUTANNAGAR',
                                               'EKKATUTHANGAL', 'City', 'CHENNAI', 'Denomination :', '2000 x', '1', 'Pincode', '600032', '500 x', '=', 'Taluk', '200 x=',
                                               '100 x', '=', 'District', 'CHENNAI', '50 x', '=', 'Mobile', '6381683601', '20 x', '10 x', '=', 'Landline', 'Total',
                                               '=', 'e-mail', 'ESAKKYMUTHU403@GMAIL.COM', '(PLEASE WRITE IN BLOCK LETTERS)', 'Name', 'SELVAGANESH.S', 'Door No', 'S 2',
                                               'Address', '2NDCROSSSTREET BALAJI', 'NAGAR ERKATUTHANGAL', 'City', 'CHENNAI', 'Denomination :', '2000 x', '-', '1',
                                               'Pincode', '600032', '500 x', 'Taluk', '200 x', '=', '100 x', 'District', 'CHENNAI', '50 x', '-', 'Mobile',
                                               '6382669478', '20 x', '10>', '-', 'Landline', 'Total', '=', 'e-mail', 'SSELVAGANESH14102000@gmail.com', '-']}


In [62]:
ocr_json2 = res2['detected_text']
ocr_json = res['detected_text']


In [63]:

all_info, df1 = process_text1(ocr_json, fields)


raw_text Name ESAKKY MUTHU.M Door No 14/11 Address ACHUTANNAGAR EKKATUTHANGAL City CHENN - Pincode 600032 Taluk District CHENNAI Mobile 6381683601 Landline e-mail ESAKKYMUTHU403@GMAIL.COM  Name SELVAGANESH.S Door No S 2 Address 2NDCROSSSTREET BALAJI NAGAR ERKATUTHANGAL City CHENNAI Pincode 600cB2 1 Taluk District CHENNAI Mobile 6382669478 Landline e-mail SSELVAGANESH14102000@gm,l.com -
records ['Name ESAKKY MUTHU.M Door No 14/11 Address ACHUTANNAGAR EKKATUTHANGAL City CHENN - Pincode 600032 Taluk District CHENNAI Mobile 6381683601 Landline e-mail ESAKKYMUTHU403@GMAIL.COM  ', 'Name SELVAGANESH.S Door No S 2 Address 2NDCROSSSTREET BALAJI NAGAR ERKATUTHANGAL City CHENNAI Pincode 600cB2 1 Taluk District CHENNAI Mobile 6382669478 Landline e-mail SSELVAGANESH14102000@gm,l.com -']
ok_email ESAKKYMUTHU403@gmail.com
gm,l
correction emial gmail


In [64]:
all_info

{0: {'Name': 'ESAKKY MUTHU.M',
  'Door No': '14/11',
  'Address': 'ACHUTANNAGAR EKKATUTHANGAL',
  'City': 'THENI',
  'Pincode': '600032',
  'District': 'CHENNAI',
  'Mobile': '6381683601',
  'Landline': '',
  'e-mail': 'ESAKKYMUTHU403@gmail.com'},
 1: {'Name': 'SELVAGANESH.S',
  'Door No': 'S 2',
  'Address': '2NDCROSSSTREET BALAJI NAGAR ERKATUTHANGAL',
  'City': 'CHENNAI',
  'Pincode': '600082',
  'District': 'CHENNAI',
  'Mobile': '6382669478',
  'Landline': '',
  'e-mail': 'SSELVAGANESH14102000@gmail.com'}}